# Course 4, Module 5, Lesson 3 Screencasts

## M5L3SC1: Transcribing SPeech to Text with Whisper

### Step 1: Setting Up Your Environment

In [ ]:
import os
os.system('pip install torch transformers torchaudio > /dev/null 2>&1')

0

### Step 2: Loading the Whisper Model

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# Load pre-trained Whisper model
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

### Step 3: Loading and Pre-processing the Audio

In [ ]:
# Load audio file
audio_path = "audio_sample.wav"  # Replace with the path to your audio file
waveform, sample_rate = torchaudio.load(audio_path)

# Convert file from stereo to mono
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

waveform = waveform.squeeze(0)

# Pre-process audio for Whisper
inputs = processor(waveform, sampling_rate=sample_rate, return_tensors="pt")

### Step 4: Transcription with Whisper

In [ ]:
# Generate transcription
output = model.generate(inputs["input_features"])
transcription = processor.batch_decode(output, skip_special_tokens=True)
print("Transcription:", transcription[0])

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription:  Dancing in the masquerade, idle truth and plain sight jaded, pop, roll, click, shot, who will I be today or not? But such a tide as moving seems asleep, too full for sound and foam, when that witch drew from out the boundless deep turns again home, twilight and evening bell and after that


### Step 5: Evaluating and Customizing the Output

In [ ]:
# Customize output
output = model.generate(inputs["input_features"], max_length=500, length_penalty=1.0, num_beams=4)
custom_transcription = processor.batch_decode(output, skip_special_tokens=True)
print("Customized Transcription:", custom_transcription[0])

Customized Transcription:  Dancing in the masquerade, idle truth and plain sight jaded, pop, roll, click, shot, who will I be today or not? But such a tide as moving seems asleep, too full for sound and foam, when that witch drew from out the boundless deep turns again home twilight and evening bell and after that


## M5L3SC2: Generating Speech from Text with TTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Step 1: Setting Up Your Environment

In [ ]:
# upgrade to a recent, compatible stack
!pip install -U "transformers[audio]" datasets>=2.19.2 fsspec>=2024.3.0 soundfile torch --quiet

import random
import torch
import soundfile as sf
from datasets import load_dataset
from transformers import pipeline, SpeechT5HifiGan

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.


### Step 2: Loading and Configuring the TTS Model

In [ ]:
# 1️. Load the SpeechT5 acoustic model + HiFi-GAN vocoder
device = "cuda" if torch.cuda.is_available() else "cpu"

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

tts = pipeline(
    task="text-to-speech",
    model="microsoft/speecht5_tts",
    vocoder=vocoder,
    device=device,
)

# 2️. Load a speaker embedding using streaming
embeddings_ds = load_dataset(
    "Matthijs/cmu-arctic-xvectors", split="validation", streaming=True
)

# Convert the iterable to a list (only if small – here we just take one sample)
first_sample = next(iter(embeddings_ds))
spk_emb = torch.tensor(first_sample["xvector"]).unsqueeze(0).to(device)

Device set to use cpu


NotImplementedError: Loading a streaming dataset cached in a LocalFileSystem is not supported yet.

### Step 3: Synthesizing Text into Speech


In [ ]:
# 3️. Synthesize speech
text = (
    "Welcome to the future of speech technology. "
    "Let's explore how text transforms into voice."
)
speech_dict = tts(text, speaker_embeddings=spk_emb)

# 4️. Save to WAV
audio, sr = speech_dict["audio"], speech_dict["sampling_rate"]
sf.write("generated_speech.wav", audio, sr)
print("Speech saved to generated_speech.wav")

ValueError: `speaker_embeddings` must be specified. For example, you can use a speaker embeddings by following
                    the code snippet provided in this link:
                    https://huggingface.co/datasets/Matthijs/cmu-arctic-xvectors
                    

### Step 4: Customizing Audio Output

In [ ]:
audio_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=5)
